In [1]:
from typing import List, Tuple

import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip

In [16]:
def time_to_seconds(time_obj: pysrt.srttime.SubRipTime) -> float:
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000

def create_subtitle_clips(
        subtitles: List[pysrt.srttime.SubRipTime],
        videosize: List[int],
        fontsize: int = 24,
        font: str = "Arial",
        color: str = "white",
        debug: bool = False
    ) -> List[TextClip]:
    subtitle_clips = []

    for subtitle in subtitles:
        start_time: float = time_to_seconds(subtitle.start)
        end_time: float = time_to_seconds(subtitle.end)
        duration: float = end_time - start_time

        video_width, video_height = videosize
        
        text_clip: TextClip = TextClip(
            subtitle.text,
            fontsize=fontsize,
            font=font,
            color=color,
            bg_color="black",
            size=(video_width*3/4, None),
            method="caption",
        ).set_start(start_time).set_duration(duration)

        subtitle_x_position: str = "center"
        subtitle_y_position: float = video_height * 4 / 5 
        text_position: Tuple[str, float] = (subtitle_x_position, subtitle_y_position)

        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips

def add_subtitle_to_movie(mp4filename: str, srtfilename) -> CompositeVideoClip:
    print("Video path:", mp4filename)
    print("Subtitle path:", srtfilename)
    video: VideoFileClip = VideoFileClip(mp4filename)
    subtitles: List[pysrt.srttime.SubRipTime] = pysrt.open(srtfilename)

    begin, _ = mp4filename.split(".mp4")
    output_video_file: str = begin + "_subtitled" + ".mp4"
    output_audio_file: str = begin + ".mp3"

    video.audio.to_audiofile(filename=output_audio_file)
    
    subtitle_clips: List[TextClip] = create_subtitle_clips(subtitles,video.size)
    
    final_video: CompositeVideoClip = CompositeVideoClip([video] + subtitle_clips)
    final_video.write_videofile(
        output_video_file,
        audio=output_audio_file,
        # temp_audiofile=output_audio_file,
        audio_codec="libmp3lame",
        remove_temp=True
        )

    return final_video

In [17]:
MP4FILENAME: str = "/Users/abevallerian/Downloads/AI_SaaS.mp4"
STRFILENAME: str = "/Users/abevallerian/Downloads/AI_SaaS.srt"

In [18]:
final_video = add_subtitle_to_movie(MP4FILENAME, STRFILENAME)

Video path: /Users/abevallerian/Downloads/AI_SaaS.mp4
Subtitle path: /Users/abevallerian/Downloads/AI_SaaS.srt
MoviePy - Writing audio in /Users/abevallerian/Downloads/AI_SaaS.mp3


MoviePy - Done.
Moviepy - Building video /Users/abevallerian/Downloads/AI_SaaS_subtitled.mp4.
Moviepy - Writing video /Users/abevallerian/Downloads/AI_SaaS_subtitled.mp4



Moviepy - Done !
Moviepy - video ready /Users/abevallerian/Downloads/AI_SaaS_subtitled.mp4


In [ ]:
video: VideoFileClip = VideoFileClip(MP4FILENAME)

In [ ]:
video.audio.to_audiofile(filename='/Users/abevallerian/Downloads/AI_SaaS_audio.mp3')

In [6]:
?final_video.write_videofile

Signature:
final_video.write_videofile(
    filename,
    fps=None,
    codec=None,
    bitrate=None,
    audio=True,
    audio_fps=44100,
    preset='medium',
    audio_nbytes=4,
    audio_codec=None,
    audio_bitrate=None,
    audio_bufsize=2000,
    temp_audiofile=None,
    rewrite_audio=True,
    remove_temp=True,
    write_logfile=False,
    verbose=True,
    threads=None,
    ffmpeg_params=None,
    logger='bar',
)
Docstring:
Write the clip to a videofile.

Parameters
-----------

filename
  Name of the video file to write in.
  The extension must correspond to the "codec" used (see below),
  or simply be '.avi' (which will work with any codec).

fps
  Number of frames per second in the resulting video file. If None is
  provided, and the clip has an fps attribute, this fps will be used.

codec
  Codec to use for image encoding. Can be any codec supported
  by ffmpeg. If the filename is has extension '.mp4', '.ogv', '.webm',
  the codec will be set accordingly, but you can still